### Step 1: Import Dependencies

1a. Clone cvat locally 
    
    git clone https://github.com/opencv/cvat

1b. Install docker


In [3]:
import fiftyone as fo
from fiftyone import ViewField as F
from client.yolov7_client.yolov7_triton_client import YoloV7_Triton_Inference_Client
from client.detectron2_client.detectron2_triton_client import Detectron2_Triton_Client
from ipywidgets import interact, Dropdown

### Step 2: Start Triton Inference Server and CVAT 

(*will likely be combined into single start up script*)

2a. Triton Inference Server: run the following command, replace source with models' path

    docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
    -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
    nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
    --log-verbose 1

    The command is as follows:
    * --gpus all: specifies to use all available GPU on device
    * --ipc=host: docker will share resource with host machine
    * --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864: set up container runtime configs
    * -p8000:8000: expose port 8000 for HTTP 
    * -p8001:8001: expose port 8001 for GRPC 
    * -p8002:8002: expose port 8002 for metrics 
    * -mount type=bind,source="path/to/triton/models",destination=/models: mount models to container
    * nvcr.io/nvidia/tritonserver:22.06-py3: pull from triton server image
    * tritonserver --model-repository=/models --strict-model-config=false \
        --log-verbose 1: starts triton inference server
    

2b. CVAT: install local server as per instructions found [here](https://opencv.github.io/cvat/docs/administration/basics/installation/)

   Once installed, to start a local CVAT server at port 8080 run the following commands

    cd /path/to/cvat/clone/ \
    docker compose up -d


In [2]:
# docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
# -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
# nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
# --log-verbose 1

### Step 3: cd to baseline directory

In [2]:
# make sure in baseline_system directory
%pwd

'c:\\Users\\Alex Lin\\Desktop\\baseline_system'

### Step 4: Start Fiftyone Instance Locally

    Run the following cell, and navigate to localhost:5151

In [2]:
# start fiftyone

session = fo.launch_app(auto=False)
session.open_tab()


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

### Step 5: Initilize the Model Client (and other things)


5a. initialize the variables associated with the model as well as the class labels 

*Note: currently all demo models are trained on coco_classes.  If a custom model implements \
separate class labels, please make sure to replace the classes with the correct class label \
for use in annotation later*

    - url: Inference server URL, default localhost:8001
    - model_info: Print model status, configuration and statistics
    - verbose: Enable verbose client output
    - client_timeout: Client timeout in seconds, default no timeout
    - ssl: Enable SSL encrypted channel to the server
    - root_certificates: File holding PEM-encoded root certificates, default none
    - private_key: File holding PEM-encoded private key, default is none
    - certificate_chain: File holding PEM-encoded certicate chain default is none
    - client_timeout: Client timeout in seconds, default no timeout
    - width: Inference model input width, default 640
    - height: Inference model input height, default 640

In [ ]:
url_triton='localhost:8001',
model_info=False,
verbose=False,
client_timeout=None,
ssl=False,
root_certificates=None,
private_key=None,
certificate_chain=None,
width=640,
height=640
# change this to the class labels your model of choice was trained on.  
# the default demo detectron2 and yolov7 labels are trained on COCO labels
COCO_CLASSES=["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","traffic light","fire hydrant",
              "stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra",
              "giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
              "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork",
              "knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut",
              "cake","chair","couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote","keyboard",
              "cell phone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear",
              "hair drier", "toothbrush"]

5b. select the desired client

*If new clients are loaded onto the triton inference server, simply append them \
to the client list and add the conditionals in init_client() to create a client. \
Make sure to import the client*

In [ ]:
# initialize the client
client_list = ["yolov7", "detectron2"]
clientW = Dropdown(options=client_list)
client = None

@interact(client_choice=clientW)
def init_client(client_choice):
    """
    Initializes client with choice from dropdown

    :params:
        - client_choice: chosen from client_list
    """
    if client_choice == "yolov7":
        client = YoloV7_Triton_Inference_Client(
            url=url_triton,
            model_info=model_info,
            verbose=verbose,
            client_timeout=client_timeout,
            ssl=ssl,
            root_certificates=root_certificates,
            private_key=private_key,
            certificate_chain=certificate_chain,
            width=width,
            height=height
        )
    elif client_choice == "detectron2":
        client = Detectron2_Triton_Client(
            url=url_triton,
            model_info=model_info,
            verbose=verbose,
            client_timeout=client_timeout,
            ssl=ssl,
            root_certificates=root_certificates,
            private_key=private_key,
            certificate_chain=certificate_chain,
            width=1344,
            height=1344
        )

5c. Set Runtime Configs 

*Make sure these configs are for your desired inference mode to avoid errors*

For images
   - input_: Input directory to load from in image
       NOTE: directory must only contain image files
   - fo_dataset: Dataset name to export predictions to fiftyone, 
       default '', no export
   - output_: Output directory, default no output saved
   - tags: list of tags to organize inference results in fiftyone

For videos
   - input_: Input directory to load from in video
       NOTE: directory must only contain video files
   - fo_dataset: Dataset name to export predictions to fiftyone, 
       default '', no export
   - output_: Output directory, default no output saved
   - fps: Video output fps, default 24.0 FPS
   - tags: list of tags to organize inference results in fiftyone

Dummy requires no input 


*Operations to export to fiftyone and export locally can both be down.  Visualization \
(not listed here) is limited and should only be used for development*

In [ ]:
input_ = ''
output_ = ''
fo_dataset = ''
fps = 24.0
tags = []

5d. Run Inference in Desired Mode

**If exported to fiftyone, navigate to your fiftyone client (should be a webpage \
at localhost:5151), refresh, and select the dataset exported to to see inference \
results**

In [ ]:
inference_choice = ['image', 'video', 'dummy']
inferenceW = Dropdown(options=inference_choice)

@interact(mode=inferenceW)
def inference(mode):
    """
    Runs inference through Triton Inference Server 

    :params:
        - mode, media type to run through server, chosen from inference_choice
    """
    if mode == 'image':
        client.infer_image(input_=input_, output_=output_, fo_dataset=fo_dataset, tags=tags)
    elif mode == 'video':
        client.infer_video(input_=input_, output_=output_, fo_dataset=fo_dataset, fps=fps, tags=tags)
    elif mode == 'dummy':
        client.infer_dummy()

### Step 6: Validation Via CVAT

Once inference has completed, validation may be performed through CVAT.  The dataset 
visualized in fiftyone would then be uploaded to CVAT, ground truth can be annotated,
and the result imported back to fiftyone for analysis.

6a. Load the desired dataset (of all available on fiftyone)

In [ ]:
datasets = fo.list_datasets()
datasetsW = Dropdown(options=datasets)
dataset = None

@interact(dataset_choice=datasetsW)
def init_dataset(dataset_choice):
    # load dataset 
    dataset = fo.load_dataset(dataset_choice)

***
**WARNING: The follow will delete the selected samples from a dataset in fiftyone, \
only run if you are for sure deleting them as they cannot be recovered**

In [ ]:
# delete selected samples
delete_view = dataset.select(session.selected)
dataset.delete_samples(delete_view)

***

6b. Config for Validation

In [ ]:
anno_key = "test_run_images"
label_field="ground_truth"
label_type='detections'
classes=COCO_CLASSES
launch_editor=True
url_cvat="http://localhost:8080"
username="django"
password="bfc"

6c. Run CVAT Validation

Specific Metrics to Pick Data to Validate, **come up with them**

In [4]:
# create specific view for low confidence model predictions
low_conf_view = (
    dataset
    .filter_labels("model_detections", F("confidence") < 0.6)
    .sort_by(F("model_detections.confidence"), reverse=True)
)

sample_id = low_conf_view.head(3)
view = dataset.select(sample_id)

# send samples to CVAT
view.annotate(
    anno_key,
    label_field=label_field,
    label_type=label_type,
    classes=classes,
    launch_editor=launch_editor,
    url=url_cvat,
    username=username,
    password=password,
)

6d. Merge Dataset Back to Fiftyone and Cleanup CVAT

In [ ]:
# merge annotations back to Fiftyone dataset
dataset.load_annotations(anno_key)
dataset.load_annotation_view(anno_key)

# Delete tasks from CVAT
results = dataset.load_annotation_results(anno_key)
results.cleanup()

# Delete run record (not the labels) from FiftyOne
dataset.delete_annotation_run(anno_key)